In [1]:
import os
import spotify

sp = spotify.login(scope='playlist-read-private playlist-modify-private user-library-read user-library-modify')
user = sp.current_user()
user['display_name']   # check if login successful



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=ca3831dc54e745ec9fd50425c059da48&response_type=code&redirect_uri=https%3A%2F%2Flocalhost%3A8080%2F&scope=playlist-modify-private+playlist-read-private+user-library-modify+user-library-read in your browser


Enter the URL you were redirected to: https://localhost:8080/?code=AQAtOAuqMHGZ8X2MMBfCXluEKKxqDqta1F5IeuzkQWZaAsBv478DdYskjFmEZcVuDUAebKybfqJkNAk6_qZej1XKMGw3Hvjs_sy_siGuQXgwBYIRlGuPktq5dHxRyg2wtdahEnngdMjILgP3CWRL5vlUJezzsmDSBDEmzmfNLcYZPNV512bNYDCDlYGLORboxyMbVi8Wh9IBwWAo3Eq-tRJ0Wqp7p-oZwBEDOpBwTxHt9BVjH_BbUmmX_MywXtPEdx513oKRULrEIhCwgaCMNnKqF6CtLajBRRNqc451qrE4HpnGFw




'Mehmet Birgi'

In [26]:
album_results = sp.current_user_saved_albums()
album_results.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [27]:
album_results['total']

710

In [28]:
albums = album_results['items']
while album_results['next']:
    album_results = sp.next(album_results)
    albums.extend(album_results['items'])
len(albums)

710

In [29]:
albums[0]['album']['total_tracks']

9

In [30]:
user_album_tracks = 0
for album in albums:
    user_album_tracks += album['album']['total_tracks']

In [31]:
user_album_tracks

8664

In [32]:
track_results = sp.current_user_saved_tracks(limit=50)  # max allowed is 50
track_results.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [33]:
track_results['total']

3451

#### delete tracks from saved tracks that are also saved as a whole album:

In [34]:
# get all tracks:
tracks = track_results['items']
while track_results['next']:
    track_results = sp.next(track_results)
    tracks.extend(track_results['items'])
print(f"fetched {len(tracks)} tracks")

fetched 3451 tracks


In [35]:
albums[0]['album'].keys()

dict_keys(['album_type', 'artists', 'available_markets', 'copyrights', 'external_ids', 'external_urls', 'genres', 'href', 'id', 'images', 'label', 'name', 'popularity', 'release_date', 'release_date_precision', 'total_tracks', 'tracks', 'type', 'uri'])

In [36]:
albums[0]['album']['tracks']['items'][0].keys()

dict_keys(['artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_urls', 'href', 'id', 'is_local', 'name', 'preview_url', 'track_number', 'type', 'uri'])

In [37]:
album = albums[0]['album']
album['artists'][0]['name']

'St Germain'

In [38]:
manual_check = []
for album in albums:
    album_tracks = album['album']['tracks']['items']
    track_ids = [track['id'] for track in album_tracks]
    if not album['album']['total_tracks'] == len(track_ids):
        results = sp.album_tracks(album['album']['id'])
        tracks = results['items']
        while results['next']:
            results = sp.next(results)
            tracks.extend(results['items'])
        track_ids = [track['id'] for track in tracks]
    try:
        assert album['album']['total_tracks'] == len(track_ids)
    except:
        print(f"Total tracks error: {album['album']['name']}' by '{album['album']['artists'][0]['name']}'\
                with id {album['album']['id']}")
        continue
#     print(f"track ids: {track_ids}")
    if len(track_ids) <= 50:
        if all(sp.current_user_saved_tracks_contains(track_ids)):
            sp.current_user_saved_tracks_delete(track_ids)
            print(f"deleted all {len(track_ids)} tracks from album '{album['album']['name']}' by '{album['album']['artists'][0]['name']}'")
    else:
        manual_check.append(album)

retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
deleted all 11 tracks from album 'What's Going On (Remastered w/Bonus Tracks)' by 'Marvin Gaye'
deleted all 14 tracks from album 'Backatown' by 'Trombone Shorty'
deleted all 14 tracks from album 'For True' by 'Trombone Shorty'
deleted all 10 tracks from album 'Say That To Say This' by 'Trombone Shorty'
deleted all 10 tracks from album 'Quartetto' by 'Quartetto Di Lucca'
deleted all 6 tracks from album 'Cosmic Funk' by 'Lonnie Liston Smith and the Cosmic Echoes'
deleted all 7 tracks from album 'Expansions' by 'Lonnie Liston Smith'
deleted all 27 tracks from album 'Alone Together: The Best Of The Mercury Years' by 'Max Roach'
deleted all 9 tracks from album 'Think Beautiful' by 'Bill Evans Trio'
deleted all 7 tracks from album 'The Ultimate Blue Train' by 'John Coltrane'
deleted all 13 tracks from album 'Wanted...Thelonious Monk' by 'Thelonious Monk'
deleted all 10 tracks from album 'Underground (Special Edition)' by

deleted all 12 tracks from album 'Excursions' by 'Eddie Harris'
deleted all 5 tracks from album 'Second Movement' by 'Eddie Harris'
deleted all 7 tracks from album 'Instant Death' by 'Eddie Harris'
deleted all 6 tracks from album 'The Electrifying Eddie Harris' by 'Eddie Harris'
deleted all 6 tracks from album 'Come On Down!' by 'Eddie Harris'
deleted all 6 tracks from album 'High Voltage' by 'Eddie Harris'
deleted all 9 tracks from album 'Silver Cycles' by 'Eddie Harris'
deleted all 8 tracks from album 'Mighty Like a Rose' by 'Eddie Harris'
deleted all 6 tracks from album 'The In Sound' by 'Eddie Harris'
deleted all 6 tracks from album 'Third Plane' by 'Ron Carter'
retrying ...1secs
deleted all 10 tracks from album 'ディア・マイルス' by 'Ron Carter'
deleted all 6 tracks from album 'Where? (RVG Edition)' by 'Ron Carter'
deleted all 5 tracks from album 'New York Slick' by 'Ron Carter'
deleted all 5 tracks from album 'Patrão' by 'Ron Carter'
deleted all 6 tracks from album 'Parade' by 'Ron Carte

In [45]:
[(album['album']['id'], album['album']['name']) for album in manual_check]

[('1Y1xgrYHw2lcDqWStVl6rK', 'Luke Cage: Season 2'),
 ('1WHJCR7JB1mydTiBJpiF67', 'Hard Bop Classics'),
 ('68EGBBqUeBKF5F4xsdKb1V', 'Handel: Agrippina'),
 ('0OtKOOIdGDNQvTIcEgDRIU', 'Handel: Tamerlano'),
 ('3hkrAa6E3GdGlfgNQPYB2I', 'Gluck: Orphée et Eurydice'),
 ('5FIaGqpxtwgcM4cGVBQMlv',
  'Gluck: Iphigénie en Tauride - The Sony Opera House'),
 ('3HZTdHBfzDGAi7fKEDghgL', '100 Best Relaxing Classics'),
 ('5ZHCqltda5ywYkHPqFIUQJ', 'J.S. Bach: Matthäus-Passion BWV 244'),
 ('2rA6ZfJR23Bg7oX9sWzTUf',
  'J.S. Bach: St Matthew Passion, BWV 244 (Matthäus-Passion)'),
 ('0wpy3RStNTnQkyOF7x9uAo', '100 Hard Bop'),
 ('6d5jkRQxNth7GmOgibfkPX',
  'The Complete Lester Young Studio Sessions On Verve'),
 ('6YQxPRto33Zn2SpzIFIobt', 'True Funk [3 CD Set]'),
 ('2Haf23oAyVyWPhMSdvrzIK', 'Handel. Rinaldo'),
 ('16RX4xwZEm8y5qpCGpA0Sb',
  'What It Is! Funky Soul And Rare Grooves [1967-1977][digital version] [International]'),
 ('3O2SUQ0VEHYYU1BKwsNV0O', 'Sun Records - 60 Years, 60 Singles Box Set'),
 ('0lcdc9pU

In [47]:
sp.current_user_saved_albums_delete(['3HZTdHBfzDGAi7fKEDghgL'])

In [ ]:
spotify:album:5uD7yoQZensEeRQow76SeW spotify:album:6d5jkRQxNth7GmOgibfkPX